# JSON interface for diagram creation
------
## Example of Entity Relationship Diagram
- [Functions used to create diagram from json](#Functions-used-to-create-diagram-from-json)
- [ER-example with simple structure](#ER-example-with-simple-structure)
- [ER-second example with additional information](#ER-second-example-with-additional-information)

## Example of Flowchart Diagram
- [FC-example with simple structure](#FC-example-with-simple-structure)
- [Functions used to convert NTV data in mermaid JSON](#Functions-used-to-convert-NTV-data-in-mermaid-JSON)
- [Example of NTV data](#Example-of-NTV-data)
- [Example of NTV-TAB data](#Example-of-NTV-TAB-data)

-------

## Example of Entity Relationship Diagram

### Functions used to create diagram from json

In [1]:
from json_ntv import Ntv

def diagram(diag):
    ntv = Ntv.obj(diag)
    diag_type = ntv.type_str[1:]
    diag_txt = '---\ntitle: ' + ntv.name + '\n---\n' if ntv.name else ''
    diag_txt += diag_type
    match diag_type:
        case 'erDiagram':
            diag_txt += erDiagram(ntv)
        case 'flowchart':
            diag_txt += flowchart(ntv)
    return diag_txt

def flowchart(ntv):
    orientation  = {'top-down' : 'TD', 'top-bottom' : 'TB','bottom-top': 'BT', 'right-left': 'RL', 'left-right': 'LR'}
    fc = Ntv.obj(ntv.val)
    diag_txt = ' ' + orientation[fc['orientation'].val]
    for node in fc['node']:
        diag_txt += fcNode(node)
    for link in fc['link']:
        diag_txt += fcLink(link)
    return diag_txt + '\n'    

def fcLink(link):
    link_t  = {'normal' : ' ---', 'normalarrow': ' -->', 'dotted': ' -.-', 'dottedarrow': ' -.->'}
    link_txt = '\n    ' + str(link[0].val) + link_t[link[1].val]
    if len(link) == 4:
        link_txt += '|' + link[3].val + '|'
    return link_txt + ' ' + str(link[2].val)

def fcNode(node):
    shape_l  = {'rectangle' : '[', 'roundedge': '(', 'stadium': '(['}
    shape_r  = {'rectangle' : ']', 'roundedge': ')', 'stadium': '])'}
    return '\n    ' + node.name + shape_l[node[0].val] + '"' + node[1].val + '"' + shape_r[node[0].val]

def erDiagram(ntv):
    diag_txt = ''
    er = Ntv.obj(ntv.val)
    for entity in er['entity']:
        diag_txt += erEntity(entity)
    for relation in er['relationship']:
        diag_txt += erRelation(relation)
    return diag_txt

def erEntity(entity):
    ent_txt = '\n    ' + entity.name + ' {'
    for att in entity:
        ent_txt += '\n        ' + att[0].val + ' ' + att[1].val
        if len(att) > 2:
            if att[2].val in ('PK', 'FK', 'UK'):
                ent_txt += ' ' + att[2].val
            else:
                ent_txt += ' "' + att[2].val + '"'
        if len(att) > 3:
            ent_txt += ' "' + att[3].val + '"'
    return ent_txt + '\n    }'

def erRelation(rel):
    rel_left  = {'exactly one' : ' ||', 'zero or one': ' |o', 'zero or more': ' }o', 'one or more': ' }|'}
    rel_right = {'exactly one' : '|| ', 'zero or one': 'o| ', 'zero or more': 'o{ ', 'one or more': '|{ '}
    identif   = {'identifying' : '--', 'non-identifying' : '..'}
    rel_txt = '\n    ' + rel[0].val + rel_left[rel[1].val] + identif[rel[2].val] + rel_right[rel[3].val] + rel[4].val
    if len(rel) > 5:
        rel_txt += ' : ' + rel[5].val
    return rel_txt

In [2]:
from base64 import b64encode
from IPython.display import Image, display

### ER-example with simple structure

In [3]:
# The Json syntax is enriched by separator ':' or '::' in dict key to indicate that:
#  - the second part of the key string is the data type
#  - the dict value is a json entity (':') or a list of Json-NTV entities.

order_example = { 
    'json order example:$erDiagram' : { 
        'relationship::': [ 
            [ 'CUSTOMER', 'exactly one', 'identifying', 'zero or more', 'ORDER',     'places'],
            [ 'ORDER',    'exactly one', 'identifying', 'one or more',  'LINE-ITEM', 'contains'] 
        ],
        'entity::': {
            'CUSTOMER':  [ 
                ['string', 'name',         'PK', 'the name'], 
                ['string', 'custNumber'] 
            ], 
            'ORDER': [ 
                ['int',    'orderNumber',  'PK'],
                ['string', 'deliveryAdress'] 
            ],
            'LINE-ITEM': [ 
                ['string', 'productCode',  'PK'],
                ['int',    'quantity'],
                ['float',  'pricePerUnit'] 
            ]   
        },
     } }

diag = diagram(order_example)
display(Image(url="https://mermaid.ink/img/" + b64encode(diag.encode("ascii")).decode("ascii")))

### ER-second example with additional information

In [41]:
# in this example we use the potential of the NTV format to add additional information
# (without changing either the code or the output data)

order_example2 = { 
    'json order example 2:$erDiagram' : { 
        'relationship::': [ 
            [ {'1st entity':'CUSTOMER'}, 'exactly one', 'identifying', 'zero or more', {'2nd entity':'ORDER'}, 'places'],
            [ 'ORDER', {'to be confirmed': 'exactly one'}, 'identifying', 'one or more',  'LINE-ITEM', {'label': 'contains'}] 
        ],
        'entity::': {
            'CUSTOMER':  [ 
                ['string', 'name', 'PK', {'comments': 'the name'}], 
                ['string', 'custNumber'] 
            ], 
            'ORDER': [ 
                {'type_att':'int', 'name_att':'orderNumber',  'key_att': 'PK'},
                {'this attribute is not yet valid' : ['string', 'deliveryAdress']}
            ],
            'LINE-ITEM': { 
                'first attribute': ['string', 'productCode',  'PK'],
                'second attribute': ['int',    'quantity'],
                'third attribute': {'type_att':'float',  'name_att':'pricePerUnit'} 
            }   
        },
     } }

diag = diagram(order_example2)
display(Image(url="https://mermaid.ink/img/" + b64encode(diag.encode("ascii")).decode("ascii")))

## Example of Flowchart Diagram

### FC-example with simple structure

In [43]:
fc_example = { 
    'json flowchart example:$flowchart' : { 
        'orientation': 'top-down',
        'node::': {
            '0' : ['roundedge', '<b>value</b>'],
            '14': ['roundedge', 'fruits'],
            '13': ['roundedge', '<b>::</b>'],
            '2' : ['rectangle', '<b>kiwi</b>\n<i>3</i>'], 
            '3' : ['rectangle', "<b>mangues</b>\n<i>4</i>"]
        },
        'link::': [ 
            [ '0' , 'normalarrow', '14'],
            [ '14', 'normalarrow', '13'],
            [ '13', 'dottedarrow', '2', 'test'],
            [ '13', 'normalarrow', '3'],         
        ]
     }
}

diag = diagram(fc_example)
display(Image(url="https://mermaid.ink/img/" + b64encode(diag.encode("ascii")).decode("ascii")))

### Functions used to convert NTV data in mermaid JSON

In [37]:
from json_ntv import NtvSingle, NtvList
import json

def mermaid_node(ntv, node, def_typ_str):
    j_name, j_sep, j_type = ntv.json_name(def_typ_str)
    name = ''
    if j_name:
        name += '<b>' + j_name + '</b>\n'
    if j_type:
        name += j_type + '\n'
    if isinstance(ntv, NtvSingle):
        if isinstance(ntv.val, str):
            name += '<i>' + ntv.val + '</i>\n'
        else:
            name += '<i>' + json.dumps(ntv.val) + '</i>\n'
        return [node, ['rectangle', name[:-1]]]
    if not name:
        name = '<b>::</b>\n'
    return [node, ['roundedge', name[:-1]]]
    
def mermaid_link(ntv, node, def_typ_str, node_list, link_list):
    node_list.append(mermaid_node(ntv, node, def_typ_str))
    if isinstance(ntv, NtvList):
        for r_node, ntv_val in enumerate(ntv):
            mermaid_link(ntv_val, node + str(r_node), ntv.type_str, node_list, link_list)
            link_list.append([node, 'normalarrow', node + str(r_node)])

# NTV is a rooted tree
def mermaid_json(ntv, title='', disp=False):
    ntv = Ntv.obj(ntv)
    node_list = []
    link_list = []
    mermaid_link(ntv, '0', None, node_list, link_list)
    mermaid_json = {title + ':$flowchart' : { 
        'orientation': 'top-down',
        'node::': {node[0]: node[1] for node in node_list},
        'link::': link_list}}
    if disp:
        return display(Image(url="https://mermaid.ink/img/" + 
                             b64encode(diagram(mermaid_json).encode("ascii")).decode("ascii")))
    return diagram(mermaid_json)

## Example of simple JSON 

In [38]:
json_example = { 'example::': {
  "fruits": [
    { "kiwis": 3,
      "mangues": 4,
      "pommes": None
    },
    { "panier": True }
  ],
  "legumes": {
      "patates": "amandine",
      "poireaux": False
    },
    "viandes": ["poisson","poulet","boeuf"]
 }}
mermaid_json(json_example, 'json flowchart', disp=True)

## Example of NTV data

In [40]:
ntv_example = { 'NTV example::': {
  "fruits": [
    { "kiwis": 3,
      "mangues:int": 4,
      "pommes": None
    },
    { "panier": True }
  ],
  "legumes::json": {
      "patates:string": "amandine",
      "poireaux": False
    },
    "viandes": ["poisson",{":string": "poulet"},"boeuf"]
 }}
diag = mermaid_json(ntv_example, 'NTV flowchart', disp=True)

## Example of NTV-TAB data

In [39]:
tab_data   = {'index':           [1, 2, 3],
              'dates::datetime': ['1964-01-01', '1985-02-05', '2022-01-21'], 
              'value':           [10, 20, 30],
              'value32::int32':  [10, 20, 30],
              'coord::point':    [[1,2], [3,4], [5,6]],
              'names::string':   ['john', 'eric', 'judith']}
diag = mermaid_json({'tab': tab_data}, 'tab data example', disp=True)